In [114]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [115]:
import nltk

In [116]:
corpus=nltk.corpus.webtext

In [117]:
sentq=[]

In [118]:
len(corpus.raw())

1726164

In [119]:
import sys

reload(sys)
sys.setdefaultencoding('utf8')

In [120]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [121]:
raw_sentences = tokenizer.tokenize(corpus.raw().decode('utf-8','ignore'))

In [122]:
raw_sentences[0]

u'Cookie Manager: "Don\'t allow sites that set removed cookies to set future cookies" should stay checked\r\nWhen in full screen mode\r\nPressing Ctrl-N should open a new browser when only download dialog is left open\r\nadd icons to context menu\r\nSo called "tab bar" should be made a proper toolbar or given the ability collapse / expand.'

In [123]:
sentq=[]

In [124]:
for sent in raw_sentences:
    if sent[-1]=='?':
        sentq.append(1)
    else: sentq.append(0)

In [125]:
sentq.count(1)*1.0/len(sentq)

0.17471084101156636

In [126]:
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [127]:
def preprocessor(desc):
    return re.sub("[^a-zA-Z]"," ", desc.lower())

In [128]:
vectorizer = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,lowercase=True,    \
                             preprocessor = preprocessor, \
                             stop_words =None,\
                             max_features = None,\
                             ngram_range=(1,1),min_df=2)   
                             #potential set binary=True
                            #potentially set preprocessor to remove all numbers                              
                            #stop words list:  list(stopwords.words("english"))  

In [129]:
def corpus_words(raw_sentences):
    corpus_bows = vectorizer.fit_transform(raw_sentences)
    #hcpcs[hcpcs.isin()]
    #cardio_bows= vectorizer.transform()
    print('Print the list of all words in the bag...')
    vocab=vectorizer.get_feature_names()
    print(vocab)
    return corpus_bows,vocab

In [130]:
corpus_bows,vocab=corpus_words(raw_sentences)

In [131]:
print('The length of vocabularies of ',len(vocab))

In [132]:
type(corpus_bows.toarray())

numpy.ndarray

In [133]:
df=pd.DataFrame(data=corpus_bows.toarray(),columns=vocab)

In [134]:
first_word=[]

In [135]:
for sent in raw_sentences:
    if sent[-1]=='?':
        first_word.append(sent.split()[0])

In [136]:
freq=nltk.FreqDist(first_word)

In [137]:
sum_freq=sum(freq.values())

In [138]:
sum_freq

4456

In [139]:
freq=dict((key.lower(),value*1.0/sum_freq) for key,value in freq.iteritems())

In [140]:
freq

{u'"?': 0.0002244165170556553,
 u'"baptism"?': 0.0002244165170556553,
 u'"gratuity"?': 0.0002244165170556553,
 u'"guys"': 0.0002244165170556553,
 u'"hey': 0.0002244165170556553,
 u'"open': 0.0002244165170556553,
 u"'cause": 0.0006732495511669659,
 u"'n'": 0.0002244165170556553,
 u'*': 0.0002244165170556553,
 u'**(*)': 0.0002244165170556553,
 u',': 0.0002244165170556553,
 u'--53rd': 0.0002244165170556553,
 u'--69th': 0.0002244165170556553,
 u'--central': 0.0002244165170556553,
 u'--college': 0.0002244165170556553,
 u'--cvs,': 0.0002244165170556553,
 u"--harry's": 0.0002244165170556553,
 u'--r': 0.0002244165170556553,
 u'--sugar': 0.0002244165170556553,
 u'--w.': 0.0002244165170556553,
 u'...': 0.0006732495511669659,
 u"..where's": 0.0002244165170556553,
 u'15-year-old': 0.0002244165170556553,
 u'18-year-old': 0.0002244165170556553,
 u'20-something': 0.0006732495511669659,
 u'3-year-old': 0.0002244165170556553,
 u'5-year-old': 0.0002244165170556553,
 u'8-year-old': 0.0004488330341113106,

In [141]:
first_word_freq=[]

In [142]:
for sent in raw_sentences:
    first_word=sent.split()[0].lower()
    print(first_word)
    if first_word in freq:
        first_word_freq.append(freq[first_word])
    else: first_word_freq.append(0)

In [143]:
sum(first_word_freq)

237.53815080792145

In [144]:
len(first_word_freq)

25505

In [145]:
df.shape

(25505, 8835)

In [146]:
df['first_word_freq']=first_word_freq

In [147]:
df.first_word_freq.describe()

count    25505.000000
mean         0.009313
std          0.015934
min          0.000000
25%          0.000224
50%          0.001122
75%          0.009201
max          0.053411
Name: first_word_freq, dtype: float64

In [148]:
from sklearn.ensemble import RandomForestClassifier

In [152]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score
from sklearn.model_selection import GridSearchCV,StratifiedShuffleSplit,cross_val_score

In [160]:

from sklearn.model_selection import train_test_split

In [167]:
def model_fit(alg, X, y, performCV=True, cv_score='recall', printFeatureImportance=True, cv=3,random_state=25):
    # function to diagnose the fit of model
    # we have precision in cross validation as the main metric, along with area under ROC, accuracy and recall.    
    # in the meanwhile, we will plot a feature importance chart
    #Fit the algorithm on the data
    X_train,X_test,y_train,y_test=train_test_split(X.as_matrix(),y,test_size=0.2,random_state=random_state)
    alg.fit(X_train, y_train)
        
    #Predict test set:
    dtrain_predictions = alg.predict(X_test)
    dtrain_predprob = alg.predict_proba(X_test)[:,1]
    
    #Perform cross-validation:
    if performCV:
        cv_score = cross_val_score(alg, X_train,y_train, cv=cv, scoring=cv_score,n_jobs=2)
    
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % accuracy_score(y_test, dtrain_predictions)
    print "Precision : %.4g" % precision_score(y_test, dtrain_predictions)
    print "Recall : %.4g" % recall_score(y_test, dtrain_predictions)
    print("Confusion Matrix (Train): ", confusion_matrix(y_test, dtrain_predictions))
    
    if performCV:
        print "CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score))
        
    #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_, X.columns).sort_values(ascending=False)[:20]
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')    
# ****************************************************************
    return feat_imp

In [154]:
cv=StratifiedShuffleSplit(n_splits=3,test_size=0.1,random_state=25)

In [155]:
seed=25
rf=RandomForestClassifier(n_jobs=-1,oob_score=True,max_features='sqrt',random_state=seed)

In [166]:
fit_model=False
if fit_model:
    model_fit(rf,df,sentq,performCV=True,cv=cv)     

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [164]:
df.head()

,aa,aaaaaah,aaaah,aaaaugh,aaagh,aaah,aaaugh,aaauugh,aagh,aah,...,zip,zipper,zippered,zippers,zone,zoo,zoom,zooming,zoot,first_word_freq
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000224
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [ ]:
param_grid_rf={'n_estimators':[50,100,300,500],'max_features':['sqrt','log2'] } #,'max_depth':[3,5,7,9],'min_samples_leaf':[0.0,0.1,0.2]}
gs_rf=GridSearchCV(rf,param_grid=param_grid_rf,n_jobs=-1,cv=cv,scoring='precision')

grid_search=True
if grid_search:
    gs_rf.fit(df.as_matrix(),sentq)
print(gs_rf.best_params_,gs_rf.best_score_)
